# Roboto SDK - Analyze Magnetic Interference in PX4 Logs

This notebook demonstrates how to use the [Roboto SDK](https://docs.roboto.ai/learn/sdk.html) to analyze a PX4 flight log to determine whether the magnetometer norm correlates with the vehicle’s thrust signal. Such correlation would indicate that current drawn by the motors or other onboard electronics is distorting the magnetic field, potentially causing incorrect yaw estimation. 

In the cells that follow, you will see how to quickly extract topics from flight logs as [pandas DataFrames](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) for analysis. For more information on any of the methods shown here, refer to the [Python SDK documentation](https://docs.roboto.ai/reference/python-sdk.html).

The data used in this notebook is public and from the PX4 Flight Review website.

## Initialization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from roboto import File

Once a log file is uploaded to Roboto, it is processed by a corresponding action that indexes the data, making it accessible through the Python SDK. In this example, we’ll work with a publicly available drone log from the PX4 Autopilot community. However, Roboto also supports other formats, such as ROS bags, MCAP files, and more.

In [ ]:
file = File.from_path_and_dataset_id(
    file_path="test_flight.ulg", 
    dataset_id="ds_4x7fa9o2s4q5"
)

## Get Magnetometer Topic Data
This cell retrieves the `sensor_mag` topic, creates a pandas DataFrame with the `x`, `y`, and `z` signals, and then plots the data.

In [ ]:
# Fetch the `sensor_mag` topic from the log file
mag_topic = file.get_topic("sensor_mag")

# Get the `x`, `y` and `z` signals from the topic as a DataFrame
mag_df = mag_topic.get_data_as_df(['x', 'y', 'z'])

# Plot the data with a single command
mag_df.plot(title="Magnetometer X, Y, Z")

Note: If you want to see the other fields (aka message paths) in the `sensor_mag` topic you can explore the record:

In [ ]:
print("Message paths in `sensor_mag` topic:")
for mp in mag_topic.message_paths:
    print(f"{mp.message_path}")

## Compute the Magnetic Field Norm
This cell calculates the magnetic field’s magnitude. Ideally, the norm remains relatively constant; large deviations may indicate external disturbances (e.g., flying near metal structures) or incorrect sensor readings. To assess this, we compare the standard deviation to the mean to see if the norm is stable within ±5% of its average.

In [ ]:
mag_norm = np.sqrt(mag_df.x**2 + mag_df.y**2 + mag_df.z**2)
mean_norm = mag_norm.mean()
std_norm  = mag_norm.std()
threshold = 0.05

if std_norm > (threshold * mean_norm):
    print("Magnetometer norm varies more than ±5% from the mean.")
else:
    print("Magnetometer norm is fairly constant within ±5% of the mean.")

mag_norm.plot(title="Magnetometer Norm", color="#9467bd")

## Get Thrust Topic Data
This cell retrieves the `vehicle_thrust_setpoint` topic data and creates a pandas DataFrame with the `x`, `y`, and `z` setpoint signals.

In [ ]:
# Fetch the `vehicle_thrust_setpoint` topic from the log file
thrust_topic = file.get_topic("vehicle_thrust_setpoint")

# Get the `xyz` signal from the topic as a DataFrame
thrust_df = thrust_topic.get_data_as_df(['xyz'])

# Expand and drop `xyz` into separate x, y, z columns
thrust_df[['x', 'y', 'z']] = thrust_df['xyz'].apply(pd.Series)
thrust_df = thrust_df.drop(columns=['xyz'])

# Plot the data with a single command
thrust_df.plot(title="Thrust X, Y, Z")

## Compute the Thrust Norm

This cell calculates the thrust vector's norm. We'll use this to compare with the magnetometer's norm.

In [ ]:
# Compute thrust vector norm
thrust_norm = np.sqrt(thrust_df.x**2 + thrust_df.y**2 + thrust_df.z**2)

thrust_norm.plot(title="Thrust Norm", color="#8c564b")

## Compare Magnetometer vs. Thrust Norm
In the plot below, note the clear correlation between the norms, which can introduce yaw estimation errors and potentially lead to a crash. To mitigate this, it is recommended to place the magnetometer (and GPS) farther from power electronics, wiring, and motors on the airframe.

In [ ]:
fig, ax = plt.subplots()
ax.plot(mag_norm, label="Magnetometer Norm", color="#9467bd")
ax.plot(thrust_norm, label="Thrust Norm", color="#8c564b")
ax.set_title("Magnetometer Norm vs. Thrust Norm")
ax.set_xlabel("Time (index)")
ax.set_ylabel("Norm")
ax.legend()
plt.show()

## Calculate Correlation 
This cell calculates the Pearson correlation between the magnetometer and thrust norms. The magnetometer data is first interpolated to match the thrust timestamps before computing the correlation coefficient.

In [ ]:
mag_norm_interp = np.interp(thrust_norm.index, mag_norm.index, mag_norm)
corr_matrix = np.corrcoef(mag_norm_interp, thrust_norm)
pearson_r = corr_matrix[0, 1]

print(f"Pearson correlation between magnetometer norm and thrust norm: {pearson_r:.4f}")